In [ ]:
import numpy as np
import scipy
from scipy.integrate import solve_ivp
import math
from scipy.fft import next_fast_len
from scipy.optimize import root_scalar
import time
import matplotlib.pyplot as plt
from numpy.fft import fft, ifft

### Value of the coupling constant between sites
gamma = 2 

### Value of the critical growth rate below which the species gets extinct at late times
rc = 0.4281423116229064

### Equilibration

def init(r, gamma, m_init, M):
    
    T = 100
    
    dt = 1e-3
    
    t_int = np.linspace(0,T,int(T/dt))
    
    traj = np.ones(M)*m_init
    
    mean_empirical = np.ones(len(t_int))*m_init

    for k in range(int(T/dt) - 1):

        ### Simulating the demographic noise alone can be done exactly, see Dornic, Chaté, Munoz, PRL (2005)

        evol_stoch_poisson = np.random.poisson(traj/dt) 

        evol_stoch = np.random.gamma(evol_stoch_poisson, scale=1.0)*dt

        ### Adding the deterministic part

        traj = evol_stoch + dt*(evol_stoch*(r - evol_stoch) + gamma*(mean_empirical[k] - evol_stoch))

        mean_empirical[k+1] = np.mean(traj)
    
    return mean_empirical, traj

### Generate trajectories from which the correlation function is computed. Only the total population size is stored

def trajectory(r, gamma, m_init, M, T):

    dt = 1e-3
    
    t_int = np.linspace(0,T,int(T/dt))

    mean_init, init_cond = init(r, gamma, m_init, M)

    traj = init_cond
    
    mean_empirical = np.ones(len(t_int))*np.mean(traj)
    
    t_init = time.time()

    for k in range(int(T/dt) - 1):

        evol_stoch_poisson = np.random.poisson(traj/dt) 

        evol_stoch = np.random.gamma(evol_stoch_poisson, scale=1.0)*dt

        traj = evol_stoch + dt*(evol_stoch*(r - evol_stoch) + gamma*(mean_empirical[k] - evol_stoch))
        
        mean_empirical[k+1] = np.mean(traj)
    
    return mean_empirical, mean_init

### Compute the two-time correlation function in a time-translation invariant regime from a long time series

def autocorr_shrinking_mean(m):
    m = np.asarray(m, dtype=np.float64)
    N = len(m)
    
    n_fft = next_fast_len(2*N)

    # FFT
    f = fft(m, n=n_fft)

    # |FFT|^2
    ac_full = np.real(ifft(f * np.conj(f)))[:N]

    # Divide by shrinking window sizes
    ac = ac_full / np.arange(N, 0, -1)

    return ac

In [ ]:
### Parameters 
### Number of sites
M = 1_000

### Total time and time step
T = 20*1_000
dt = 1e-3

### Possible values of dr = r - rc and the associated average population size, solution of Eq. (12) of the SM
dr_vec = np.array([0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5])
m_vec = np.array([0.00115022, 0.00575033, 0.0114987, 0.0574178, 0.114649, 0.228567, 0.341792, 0.454358, 0.566299])

In [ ]:
### Example run -- with initialization at the expected mean population size

k = 5

t_init = time.time()

mean_traj, mean_init_traj = trajectory(rc + dr_vec[k], gamma, m_vec[k], M, T)

t_fin = time.time()

C = autocorr_shrinking_mean(mean_empirical)

In [ ]:
### Plotting the results

k = 5
m = m_vec[k]

plt.plot(t_int[:200_000]*m, C[:200_000]*M, linewidth = 2, label = rf'$\bar{{m}} = {m:.3f}\,\,,\,M={M}$')

plt.xlim(0,10)
plt.ylim(-0.1,1.62)

plt.xticks(fontsize = 16)
plt.yticks([0, 0.4, 0.8, 1.2, 1.6], fontsize = 16)

plt.xlabel(r'$\bar{{m}}\tau$', fontsize = 18)
plt.ylabel(r'$\left\langle \delta m(\tau)  \delta m(0)\right\rangle$', fontsize = 18)
plt.legend(fontsize = 14)

plt.tight_layout()
plt.show()